# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 31 2022 2:04PM,238465,10,MET210578,"Methapharm, Inc.",Released
1,May 31 2022 2:04PM,238465,10,MET210642,"Methapharm, Inc.",Released
2,May 31 2022 1:57PM,238464,10,SOIVA0000481,Ivaoes Animal Health,Released
3,May 31 2022 1:56PM,238463,10,SONSB0001804,"Nextsource Biotechnology, LLC",Released
4,May 31 2022 1:56PM,238463,10,SONSB0001805,"Nextsource Biotechnology, LLC",Released
5,May 31 2022 1:56PM,238463,10,SONSB0001806,"Nextsource Biotechnology, LLC",Released
6,May 31 2022 1:54PM,238462,19,GP06032022,"Granules Pharmaceuticals, Inc.",Released
7,May 31 2022 1:52PM,238461,12,U0927,"Uniderm USA, Inc",Released
8,May 31 2022 1:52PM,238461,12,U0928,"Uniderm USA, Inc",Released
9,May 31 2022 1:52PM,238460,10,0085727376,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
56,238461,Released,2
57,238462,Released,1
58,238463,Released,3
59,238464,Released,1
60,238465,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238461,NaN,NaN,2.0
238462,NaN,NaN,1.0
238463,NaN,NaN,3.0
238464,NaN,NaN,1.0
238465,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238317,0.0,9.0,1.0
238334,4.0,6.0,4.0
238341,0.0,0.0,35.0
238342,0.0,0.0,1.0
238343,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238317,0,9,1
238334,4,6,4
238341,0,0,35
238342,0,0,1
238343,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238317,0,9,1
1,238334,4,6,4
2,238341,0,0,35
3,238342,0,0,1
4,238343,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238317,,9,1
1,238334,4,6,4
2,238341,,,35
3,238342,,,1
4,238343,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 31 2022 2:04PM,238465,10,"Methapharm, Inc."
2,May 31 2022 1:57PM,238464,10,Ivaoes Animal Health
3,May 31 2022 1:56PM,238463,10,"Nextsource Biotechnology, LLC"
6,May 31 2022 1:54PM,238462,19,"Granules Pharmaceuticals, Inc."
7,May 31 2022 1:52PM,238461,12,"Uniderm USA, Inc"
9,May 31 2022 1:52PM,238460,10,ISDIN Corporation
22,May 31 2022 1:51PM,238447,15,"Virtus Pharmaceuticals, LLC"
40,May 31 2022 1:49PM,238459,10,ISDIN Corporation
41,May 31 2022 1:48PM,238458,20,Alumier Labs Inc.
46,May 31 2022 1:39PM,238456,102,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 31 2022 2:04PM,238465,10,"Methapharm, Inc.",,,2
1,May 31 2022 1:57PM,238464,10,Ivaoes Animal Health,,,1
2,May 31 2022 1:56PM,238463,10,"Nextsource Biotechnology, LLC",,,3
3,May 31 2022 1:54PM,238462,19,"Granules Pharmaceuticals, Inc.",,,1
4,May 31 2022 1:52PM,238461,12,"Uniderm USA, Inc",,,2
5,May 31 2022 1:52PM,238460,10,ISDIN Corporation,,,13
6,May 31 2022 1:51PM,238447,15,"Virtus Pharmaceuticals, LLC",,,18
7,May 31 2022 1:49PM,238459,10,ISDIN Corporation,,,1
8,May 31 2022 1:48PM,238458,20,Alumier Labs Inc.,,,5
9,May 31 2022 1:39PM,238456,102,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 2:04PM,238465,10,"Methapharm, Inc.",2,,
1,May 31 2022 1:57PM,238464,10,Ivaoes Animal Health,1,,
2,May 31 2022 1:56PM,238463,10,"Nextsource Biotechnology, LLC",3,,
3,May 31 2022 1:54PM,238462,19,"Granules Pharmaceuticals, Inc.",1,,
4,May 31 2022 1:52PM,238461,12,"Uniderm USA, Inc",2,,
5,May 31 2022 1:52PM,238460,10,ISDIN Corporation,13,,
6,May 31 2022 1:51PM,238447,15,"Virtus Pharmaceuticals, LLC",18,,
7,May 31 2022 1:49PM,238459,10,ISDIN Corporation,1,,
8,May 31 2022 1:48PM,238458,20,Alumier Labs Inc.,5,,
9,May 31 2022 1:39PM,238456,102,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 2:04PM,238465,10,"Methapharm, Inc.",2,,
1,May 31 2022 1:57PM,238464,10,Ivaoes Animal Health,1,,
2,May 31 2022 1:56PM,238463,10,"Nextsource Biotechnology, LLC",3,,
3,May 31 2022 1:54PM,238462,19,"Granules Pharmaceuticals, Inc.",1,,
4,May 31 2022 1:52PM,238461,12,"Uniderm USA, Inc",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 2:04PM,238465,10,"Methapharm, Inc.",2.0,NaN,NaN
1,May 31 2022 1:57PM,238464,10,Ivaoes Animal Health,1.0,NaN,NaN
2,May 31 2022 1:56PM,238463,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
3,May 31 2022 1:54PM,238462,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
4,May 31 2022 1:52PM,238461,12,"Uniderm USA, Inc",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 2:04PM,238465,10,"Methapharm, Inc.",2.0,0.0,0.0
1,May 31 2022 1:57PM,238464,10,Ivaoes Animal Health,1.0,0.0,0.0
2,May 31 2022 1:56PM,238463,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
3,May 31 2022 1:54PM,238462,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
4,May 31 2022 1:52PM,238461,12,"Uniderm USA, Inc",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5721921,464.0,9.0,11.0
102,953753,3.0,1.0,0.0
12,953512,77.0,1.0,0.0
15,1192083,23.0,9.0,0.0
18,1430263,6.0,0.0,0.0
19,476916,2.0,0.0,0.0
20,1668931,67.0,13.0,4.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5721921,464.0,9.0,11.0
1,102,953753,3.0,1.0,0.0
2,12,953512,77.0,1.0,0.0
3,15,1192083,23.0,9.0,0.0
4,18,1430263,6.0,0.0,0.0
5,19,476916,2.0,0.0,0.0
6,20,1668931,67.0,13.0,4.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,464.0,9.0,11.0
1,102,3.0,1.0,0.0
2,12,77.0,1.0,0.0
3,15,23.0,9.0,0.0
4,18,6.0,0.0,0.0
5,19,2.0,0.0,0.0
6,20,67.0,13.0,4.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,464.0
1,102,Released,3.0
2,12,Released,77.0
3,15,Released,23.0
4,18,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,18,19,20
Status,,,,,,,
Completed,11.0,0.0,0.0,0.0,0.0,0.0,4.0
Executing,9.0,1.0,1.0,9.0,0.0,0.0,13.0
Released,464.0,3.0,77.0,23.0,6.0,2.0,67.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,18,19,20
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,4.0
1,Executing,9.0,1.0,1.0,9.0,0.0,0.0,13.0
2,Released,464.0,3.0,77.0,23.0,6.0,2.0,67.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,18,19,20
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,4.0
1,Executing,9.0,1.0,1.0,9.0,0.0,0.0,13.0
2,Released,464.0,3.0,77.0,23.0,6.0,2.0,67.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()